In [2]:
from langchain_community.chat_message_histories import ChatMessageHistory
from app.chroma_store import get_chroma_db
from app.agent import build_agent
from langchain_openai import ChatOpenAI
from app.config import LLM_MODEL, OPENAI_API_KEY, OPENAI_API_BASE, TEMPERATURE
from langchain_core.output_parsers import JsonOutputKeyToolsParser
from langchain_core.runnables import RunnablePassthrough, RunnableWithMessageHistory, RunnableSerializable
import importlib
import prompts  # the whole folder
importlib.reload(prompts)

from prompts import llm_judge_prompt
from prompts import search_judge_prompt

In [3]:
store = {}

def get_session_history(session_id: str):
    return store.setdefault(session_id, ChatMessageHistory())

In [4]:
db = get_chroma_db()
retriever = db.as_retriever()
qa_agent = build_agent(retriever, get_session_history)

In [14]:
query = 'Who founded AC/DC?'
session_id = '1'

In [15]:
response = qa_agent.invoke(
            {"input": query},
            config={"configurable": {"session_id": session_id}}
        )

print(response['output'])



> Entering new AgentExecutor chain...

Invoking: `get_band_information` with `{'query': 'Who founded AC/DC?'}`


{'context': '[Band: Ac Dc] [Source:Wikipedia] AC/DC were founded by brothers Angus (lead guitar) and Malcolm Young (rhythm guitar), with Colin Burgess (drums), Larry Van Kriedt (bass guitar) and Dave Evans (lead vocals). They underwent several line-up changes before releasing their debut Australasian-only album, High Voltage (1975). Membership stabilised after the release of Let There Be Rock (1977), with the Young brothers, Phil Rudd on drums, Cliff Williams on bass guitar and Bon Scott on lead vocals. Seven months after the release of Highway to Hell (1979), Scott died of alcohol poisoning and English singer Brian Johnson was then recruited as their new frontman. Their first album with Johnson, Back in Black (1980), dedicated to Scott\'s memory, became the second best-selling album of all time. Their eighth studio album, For Those About to Rock (1981), was their first al

In [16]:
def format_tool_calls(intermediate_steps):
    tool_calls = []
    for step, _ in intermediate_steps:
        if hasattr(step, 'tool') and hasattr(step, 'tool_input'):
            tool_calls.append({"tool": step.tool, "tool_input": step.tool_input})
    return tool_calls

tool_calls_summary = format_tool_calls(response['intermediate_steps'])

print(tool_calls_summary)

[{'tool': 'get_band_information', 'tool_input': {'query': 'Who founded AC/DC?'}}]


In [26]:
judge_llm = ChatOpenAI(
        model=LLM_MODEL,
        api_key=OPENAI_API_KEY,
        base_url=OPENAI_API_BASE,
        temperature=TEMPERATURE
    )

judge_chain = search_judge_prompt | judge_llm 

In [27]:
judge_response = judge_chain.invoke({
    "query": query,
    "final_answer": response['output'],
    "tool_calls": tool_calls_summary,
    "chat_history": store[session_id].messages
})


In [28]:
import json

raw_content = judge_response.content

# Clean it: remove ```json and ``` if present
cleaned_content = raw_content.strip()
if cleaned_content.startswith("```json"):
    cleaned_content = cleaned_content[len("```json"):].strip()
if cleaned_content.startswith("```"):
    cleaned_content = cleaned_content[len("```"):].strip()
if cleaned_content.endswith("```"):
    cleaned_content = cleaned_content[:-3].strip()

result = json.loads(cleaned_content)
print(result['should_search'])

False


In [43]:
import bandwiki_graph
importlib.reload(bandwiki_graph.graph)
from bandwiki_graph.graph import build_bandwiki_graph

bandwiki_chain = build_bandwiki_graph()

user_query = "Who founded AC/DC?"

initial_state = {
    "query": user_query,
    "chat_history": store[session_id].messages,
    "band_agent": qa_agent,
    "judge_chain": judge_chain,
}
final_state = bandwiki_chain.invoke(
    initial_state,
    config={"configurable": {"session_id": session_id}}
)


print("Final Answer:", final_state["final_answer"])




> Entering new AgentExecutor chain...
AC/DC was founded in Sydney, Australia in 1973 by brothers Angus and Malcolm Young. Other early members included Colin Burgess, Larry Van Kriedt, and Dave Evans. This information is according to Wikipedia.


> Finished chain.
Final Answer: AC/DC was founded in Sydney, Australia in 1973 by brothers Angus and Malcolm Young. Other early members included Colin Burgess, Larry Van Kriedt, and Dave Evans. This information is according to Wikipedia.

